In [35]:
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.callbacks import Callback


class LRFinder(Callback):
    """`Callback` that exponentially adjusts the learning rate after each training batch between `start_lr` and
    `end_lr` for a maximum number of batches: `max_step`. The loss and learning rate are recorded at each step allowing
    visually finding a good learning rate as per https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html via
    the `plot` method.
    """

    def __init__(self, start_lr: float = 1e-7, end_lr: float = 1, max_steps: int = 1000, smoothing=0.9):
        super(LRFinder, self).__init__()
        self.start_lr, self.end_lr = start_lr, end_lr
        self.max_steps = max_steps
        self.smoothing = smoothing
        self.step, self.best_loss, self.avg_loss, self.lr = 0, 0, 0, 0
        self.lrs, self.losses = [], []

    def on_train_begin(self, logs=None):
        self.step, self.best_loss, self.avg_loss, self.lr = 0, 0, 0, 0
        self.lrs, self.losses = [], []

    def on_train_batch_begin(self, batch, logs=None):
        self.lr = self.exp_annealing(self.step)
        tf.keras.backend.set_value(self.model.optimizer.lr, self.lr)

    def on_train_batch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        step = self.step
        if loss:
            self.avg_loss = self.smoothing * self.avg_loss + (1 - self.smoothing) * loss
            smooth_loss = self.avg_loss / (1 - self.smoothing ** (self.step + 1))
            self.losses.append(smooth_loss)
            self.lrs.append(self.lr)

            if step == 0 or loss < self.best_loss:
                self.best_loss = loss

            if smooth_loss > 4 * self.best_loss or tf.math.is_nan(smooth_loss):
                self.model.stop_training = True

        if step == self.max_steps:
            self.model.stop_training = True

        self.step += 1

    def exp_annealing(self, step):
        return self.start_lr * (self.end_lr / self.start_lr) ** (step * 1. / self.max_steps)

    def plot(self):
        fig, ax = plt.subplots(1, 1)
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate (log scale)')
        ax.set_xscale('log')
        ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.0e'))
        ax.plot(self.lrs, self.losses)

In [44]:
from models import mobilenet
clf = mobilenet.create_model(input_shape = (64,64,3))

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [45]:
from data_loader.gtsrb_template_loader import GTSRB_Template_DataGenerator
from data_loader.tt100k_template_loader import TT100K_Template_DataGenerator
train_datagen = GTSRB_Template_DataGenerator(n_way=43,k_shot=1,n_query=50,batch=128,data_type='all',target_size=(64,64),augmentation=True)
val_datagen = TT100K_Template_DataGenerator(n_way=36,k_shot=1,n_query=50,batch=128,data_type='all',target_size=(64,64))

In [46]:
def loss_fun(y_true,y_pred):
    x = -tf.math.log(y_pred + 1e-12)
    return tf.reduce_mean(y_true*x)

In [47]:
lr_finder = LRFinder()

In [48]:
from tensorflow import keras
from tensorflow.keras.metrics import CategoricalAccuracy
train_metric = CategoricalAccuracy()
optimizer = keras.optimizers.Adam(learning_rate=1.0,epsilon=1.0e-8)
#optimizer = keras.optimizers.SGD(learning_rate=args.lr,momentum=0.9)
train_loss_tracker = keras.metrics.Mean(name='train_loss')
clf.compile(optimizer=optimizer,loss=loss_fun,metrics=train_metric)

In [ ]:
clf.fit(train_datagen,epochs=1,callbacks=[lr_finder])

222/306 [====================>.........] - ETA: 14s - loss: 0.4305 - categorical_accuracy: 0.2044

In [ ]:
lr_finder.plot()

In [107]:
lr_finder.lrs[np.where(np.round(lr_finder.losses,1) == 0.2)[0][30]]

4.055085354483839e-05

In [17]:
import numpy as np

In [96]:
np.where(np.round(lr_finder.losses,1) == 0.1)

(array([ 310,  311,  312,  313,  314,  315,  316,  317,  318,  319,  320,
         321,  322,  323,  324,  325,  326,  327,  328,  329,  330,  331,
         332,  333,  334,  335,  336,  337,  338,  339,  340,  341,  342,
         343,  934,  935,  936,  937,  938,  939,  940,  941,  942,  943,
         944,  945,  946,  947,  948,  949,  950,  951,  952,  953,  954,
         955,  956,  957,  958,  959,  960,  961,  962,  963,  964,  965,
         966,  967,  968,  969,  970,  971,  972,  973,  974,  975,  976,
         977,  978,  979,  980,  981,  982,  983,  984,  985,  986,  987,
         988,  989,  990,  991,  992,  993,  994,  995,  996,  997,  998,
         999, 1000], dtype=int64),)

In [94]:
loss,acc = clf.evaluate(val_datagen)

[0.33817973732948303,
 0.3533873918809389,
 0.35814583070603684,
 0.3609086167209194,
 0.36090319757583694,
 0.3591608443446838,
 0.35559337200857505,
 0.3527240910087585,
 0.3494280388661613,
 0.3471104651185252,
 0.34551547553338313,
 0.3436954799065329,
 0.3428566279175859,
 0.3426399540774288,
 0.34222741348000263,
 0.34243482501013556,
 0.3423485290657568,
 0.34207417186304506,
 0.34217983567327925,
 0.3418277778598562,
 0.3415852149527821,
 0.3416083591247597,
 0.3417372272364325,
 0.3419178205442153,
 0.3417664434303121,
 0.34173566618362705,
 0.3418070570267703,
 0.3418134295167916,
 0.34176384707552904,
 0.34161216733177796,
 0.3416395010657744,
 0.3417038101010756,
 0.3417735440715891,
 0.3418488321207574,
 0.3420829166377933,
 0.34234476100240396,
 0.3425580870023376,
 0.34284992290326505,
 0.3432686153413303,
 0.34387689092625884,
 0.34439567162446744,
 0.344888532990783,
 0.34526981715321003,
 0.34550308727525636,
 0.3456867892231056,
 0.34596192224106753,
 0.3460210754632

In [1]:
from tensorflow.keras.applications import  MobileNet

In [4]:
mbilenet = MobileNet(input_shape=(64,64,3),weights='imagenet',include_top=False)

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet/mobilenet_1_0_224_tf_no_top.h5: None -- [WinError 10061] No connection could be made because the target machine actively refused it